In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import re
import wget
import pandas as pd
import xml.etree.ElementTree as ET
import chardet
from pprint import pprint
from bs4.element import NavigableString, Tag

# depuis internet #

In [ ]:
page = requests.get('http://cfpp2000.univ-paris3.fr/Corpus.html')
soup = BeautifulSoup(page.content, 'html') # If this line causes an error, run 'pip install html5lib' or install html5lib

In [ ]:
li = soup.find_all('li')

enregistrements = [x.get_text() for x in li if re.search(r'\xa0CFPP2000 .*', x.get_text())]

In [ ]:
trans = soup.find_all('li',{'class':'linopucetransc'})
urls = ['http://cfpp2000.univ-paris3.fr'+a['href'][1:] for x in trans for a in x.find_all('a')]

In [ ]:
wav = soup.find_all('li',{'class':'linopuceaudio'})
urls_wav = ['http://cfpp2000.univ-paris3.fr'+a['href'][1:] for x in wav for a in x.find_all('a')]

In [ ]:
def url2file(url):
    #try:
    path = r'E:/Corpus/CFPP'
    wget.download(url,out = path)
    print('téléchargé : {}'.format(url))

In [ ]:
for url in urls_wav:
    print(url)
    url2file(url)

In [ ]:
for url in urls:
    url2file(url)

# get all files #

In [ ]:
def get_all_folder(path):
    directory = os.listdir(path)
    folder_list = []
    for folder in directory:
        if os.path.isdir(os.path.join(path, folder)):
            folder_list.append(folder)
    return folder_list

In [ ]:
def add_path2corpus(path,corpus):
    new = '{}/{}'.format(path, corpus)
    return new

In [ ]:
path = 'F:/Corpus/cfpp2000/cfpp2000/4'

In [ ]:
corpus = [add_path2corpus(path,x) for x in get_all_folder(path)]

In [ ]:
enregistrements = [add_path2corpus(x,y) for x in corpus for y in get_all_folder(x)]

In [ ]:
def get_corpus_files(path):
    files = {}
    for file in os.listdir(path):
        if file.endswith('.xml'):
            files['xml']=file
        elif file.endswith('.trs'):
                files['trs']=file
        elif file.endswith('.wav'):
            files['wav']=file
        else:
            continue
    return files

def get_files_cfpp(list_corpus_path):
    mydict = {}
    for corpus_path in list_corpus_path:
        dict_corpus = get_corpus_files(corpus_path)
        mydict[corpus_path]=dict_corpus
    df = pd.DataFrame.from_dict(mydict, orient='index').rename_axis('corpus').reset_index()
    return df

In [ ]:
df = get_files_cfpp(enregistrements)

# parse xml #

In [ ]:
def get_all_tagsname(xml_file):
    print(xml_file)
    # Parse the XML file into an ElementTree object
    #try : 
    tree = ET.parse(xml_file)
# Get the root element of the tree
    root = tree.getroot()
# Initialize an empty set to store the tag names
    tag_names = set()
# Iterate over all elements in the tree and add their tag names to the set
    for elem in root.iter():
        tag_names.add(elem.tag)
    # Return the set of tag names
#     except: 
#         return "Pas de fichier trouvé à cet emplacement"
    return tag_names

In [ ]:
def get_corpus_name(path):
    filename = path.replace('.xml','').replace('E:/Corpus/cfpp2000/cfpp2000/4/','')
    return filename

In [ ]:
def get_df_tags(xml_files):
    corpus_name = [get_corpus_name(xml_file) for xml_file in xml_files]
    tags_ = [get_all_tagsname(xml_file) for xml_file in xml_files]
    df = pd.DataFrame(list(zip(corpus_name, tags_)), columns = ['corpus_name','tags'])
    return df

In [ ]:
xml_path = [str(x+'/'+y) for x,y in list(zip(df['corpus'].tolist(),df['xml'].tolist())) if isinstance(y,str)]
df_tags = get_df_tags(xml_path)

In [ ]:
def detect_encoding(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding'] 
    # Open the file with the detected encoding
    with open(file, 'r', encoding=encoding) as f:
        soup = BeautifulSoup(f, "lxml-xml")
    return soup

In [ ]:
def parse_xml(file):
    print(file)
    soup = detect_encoding(file)
    pp = soup.find('element',{'type':'Enregistrement'}).find_all('p')
    dicti = {}
    for p in pp:
        pattern = r'(.*) :(.*)'
        match = re.search(pattern, p.get_text())
        if match:
            nom = match.group(1).lower()
            valeur = match.group(2)
            dicti['corpus']=file.replace('E:/Corpus/cfpp2000/cfpp2000/4/','').replace('.xml','')
            dicti['{}'.format(nom)]=valeur
    dicti['locuteur'] = get_locuteurs(soup.find_all('description')[-1])
    print('______________________________')
    print('______________________________')
    return dicti

In [ ]:
def get_df(my_dict):
    df_corpus = pd.DataFrame([my_dict])
    return df_corpus

In [ ]:
def get_locuteurs(soup):
    enregistrement = soup.find('element',{'type':'Enregistrement'})
    next_ = enregistrement.find_next_sibling()
    dict_loc = {}
    i = 0
    while next_ is not None:
        pp = next_.find_all('p')
        dictii = {}
        for ind, p in enumerate(pp):
            pattern = r'(.*) :(.*)'
            match = re.search(pattern, p.get_text())
            if match:
                nom = match.group(1).lower()
                valeur = match.group(2)
                dictii['{}'.format(nom)]=valeur
                pattern2 = r'prénom et nom.*'
                match2 = re.search(pattern2, nom)
                if match2 and valeur:
                    nom_loc  = valeur
                else:
                    nom_loc = str(i)
                dict_loc['{}'.format(nom_loc)]=dictii
        next_ = next_.find_next_sibling()
        i +=1
    return dict_loc

In [ ]:
def get_info(liste_path, df):
    list_df =  []
    for xml in liste_path:
        dicti = parse_xml(xml)
        df_corpus = get_df(dicti)
        pattern = r'(.*)/.*$'
        match = re.search(pattern, xml)
        if match:
            folder = match.group(1)
            df_corpus['trs']=extract_trs_file_name(folder, df)
        else:
            df_corpus['trs']='NA'
        list_df.append(df_corpus)
    df = pd.concat(list_df)
    return df

In [ ]:
import os
import glob

def extract_trs_file_name(folder_path, df):
    row = df.loc[df['corpus'] == '{}'.format(folder_path)]
    trs = row['trs'].values
    return trs

In [ ]:
df_meta = get_info(xml_path, df)

# parse trs #

In [ ]:
def parse_trs2(file):
    soup = detect_encoding(file)
    my_dict = {}
    dict_trans, audio_filename = get_trans2(soup, my_dict)
    my_dict.update(dict_trans)  
    dict_loc = get_speaker2(soup)
    my_dict.update(dict_loc)
    my_dict.update(get_topics2(soup, my_dict))
    my_dict.update(get_section2(soup, my_dict))
    dict_turns = get_turn2(soup)
    print(dict_turns)
    df = pd.DataFrame.from_dict(dict_turns,orient='index')
    df['audio_filename']= audio_filename
    df['trs_filename']=file
    return df

In [ ]:
def get_trans2(soup, my_dict):
    trans = soup.find('Trans')
    # create a dictionary of locuteurs
    if trans is not None:
        audio_filename = trans.get('audio_filename')
        my_dict['audio_filename'] = audio_filename
    return my_dict, audio_filename

def get_speaker2(soup):
    speakers = soup.find_all('Speaker')
    # create a dictionary of speakers
    my_dict = {}
    for speaker in speakers:
        scope = speaker.get('scope')
        accent = speaker.get('accent')
        dialect = speaker.get('dialect')
        check = speaker.get('check')
        name = speaker.get('name')
        id_ = speaker.get('id')
        # create a new dictionary for this speaker
        dict_locuteur = {}
        dict_locuteur['scope'] = scope
        dict_locuteur['accent'] = accent
        dict_locuteur['dialect'] = dialect
        dict_locuteur['check'] = check
        dict_locuteur['name'] = fix_encoding(name)
        dict_locuteur['id'] = id_
        my_dict[id_] = dict_locuteur
    return my_dict

def get_topics2(soup, my_dict):
    topics = soup.find_all('Topic')
    # create a dictionary of locuteurs
    for topic in topics:
        id_ = topic.get('id')
        desc = topic.get('desc')
        # create a new dictionary for this speaker
        dict_topic = {}
        dict_topic['desc'] = desc
        dict_topic['id'] = id_
        my_dict[id_] = dict_topic
    return my_dict

def get_section2(soup, my_dict):
    sections = soup.find_all('Section')
    # create a dictionary of locuteurs
    for section in sections:
        topic = section.get('topic')
        endTime = section.get('endTime')
        startTime = section.get('startTime')
        type_ = section.get('type')
        # create a new dictionary for this speaker
        dict_section = {}
        dict_section['topic'] = topic
        dict_section['endTime'] = endTime
        dict_section['startTime'] = startTime
        dict_section['type'] = type_
        my_dict['section_{}'.format(topic)] = dict_section
    return my_dict

def get_turn2(soup):
    turns = soup.find_all('Turn')
    dict_turns = {}
    dict_loc = get_speaker2(soup)
    results = []
    i = 0
    for index, turn in enumerate(turns):
        speaker = turn.get('speaker')
        if speaker is not None:
            speakers_ = r'([a-zA-Z]+([0-9]+|)) ([a-zA-Z]+([0-9]+|)) *([a-zA-Z]*([0-9]*|))'
            match = re.search(speakers_, speaker)
            if match :
                print('MULTI SPEAKER')
                print(turn)
                spk1 = match.group(1)
                spk2 = match.group(3)
                liste_speaker = [spk1, spk2]
                try:
                    spk3 = match.group(3)
                    liste_speaker.append(spk3)
                except:
                    pass
                result = get_sync_multiple_loc2(turn, liste_speaker)
                for spk,text in result:
                    print('spk:', spk, 'text: ', text)
                    endTime = turn.get('endTime')
                    startTime = turn.get('startTime')
                    dict_turn = {}
                    dict_turn['startTime']=startTime
                    print(type(startTime))
                    dict_turn['endTime']=endTime
                    dict_turn['speaker'] = spk
                    dict_turn['speaker_name'] = dict_loc['{}'.format(spk)]['name']
                    dict_turn['speaker_native'] = dict_loc['{}'.format(spk)]['dialect']
                    dict_turn['text_synchronisé'] = str(((startTime, fix_encoding(text).replace('\n',''))))
                    dict_turn['text']= fix_encoding(text).replace('\n','')
                    dict_turns['{}'.format(i)] = dict_turn
                    print('_________________')
                    i+=1

                #prendre le texte de speaker 1 et le texte de speaker 2 AVEC **ALERT**
                # créer un tour par locuteur avec le même start et end time
            else: 
                print('UN SEUL SPEAKER')
                print(turn)
                endTime = turn.get('endTime')
                startTime = turn.get('startTime')
                sync_tags = turn.find_all('Sync')
                result = get_sync_times2(turn)
                text = ' '.join([content for _, content in result])
                dict_turn = {}
                dict_turn['startTime']=startTime
                dict_turn['endTime']=endTime
                dict_turn['speaker'] = speaker
                dict_turn['speaker_name'] = dict_loc['{}'.format(speaker)]['name']
                dict_turn['speaker_native'] = dict_loc['{}'.format(speaker)]['dialect']
                #dict_turn['speaker_characteristic'] = dict_loc.get('{}'.format(speaker))
                dict_turn['text']= fix_encoding(text).replace('\n', ' ').replace('  ', ' ')
                dict_turn['text_synchronisé'] = str(result).strip('[]')
                dict_turns['{}'.format(i)] = dict_turn
                print('spk: ', speaker, 'text : ', fix_encoding(text))
                print('___________________')
                i+=1
        else:
            endTime = turn.get('endTime')
            startTime = turn.get('startTime')
            dict_turn = {}
            dict_turn['startTime']=startTime
            dict_turn['endTime']=endTime
            dict_turn['speaker']= turn.text.replace('\n','')+'***NO SPEAKER***'
            dict_turn['speaker_name']= turn.text.replace('\n','')+'***NO SPEAKER***'
            dict_turns['{}'.format(i)] = dict_turn
            print('No spk', 'action : ',turn.text.replace('\n','')+'***ACTIONS***' )
            print('---')
            i+=1
    return dict_turns

In [ ]:
def fix_encoding(text):
    replacements = {
        "Ã¨": "è", # Add more replacements for other problematic characters here
        "Ã¡": "á",
        "Ã¢": "â",
        "Ã£": "ã",
        "Ã¤": "ä",
        "Ã§": "ç",
        "Ã©": "é",
        "Ãª": "ê",
        "Ã«": "ë",
        "Ã­": "í",
        "Ã¯": "ï",
        "Ã±": "ñ",
        "Ã³": "ó",
        "Ã´": "ô",
        "Ã¶": "ö",
        "Ãº": "ú",
        "Ã¼": "ü",
        "Ã½": "ý",
        "Ã¦": "æ",
        "Ã¸": "ø",
        "ÃŸ": "ß",
        "Ã" : "à",
        "à¹":'ù',
        "à»":"û",
        "\xa0":"",
        "Â": "",   # Remove non-breaking space
        "â": "-", # Replace en dash
        "â": "--" # Replace em dash
        }

    
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

In [ ]:
def get_sync_times2(turn):
    sync_tags = turn.find_all('Sync')
    list_sync = []
    for i, sync_tag in enumerate(sync_tags):
        contenu = []
        current_tag = sync_tag.next_sibling
        while current_tag is not None and current_tag.name != 'Sync':
            if isinstance(current_tag, NavigableString):
                current_tag_str = current_tag.get_text().replace('\n','')
                contenu.append(fix_encoding(current_tag_str))
            elif current_tag.name == 'Event' and current_tag.get('desc') == 'rire':
                next_sync = current_tag.find_next_sibling('Sync')
                if next_sync:
                    current_tag_str = '*rires*'
                    contenu.append(fix_encoding(current_tag_str))
            current_tag = current_tag.next_sibling
        if contenu:
            list_sync.append((sync_tag['time'], ' '.join(contenu)))
    return list_sync


In [ ]:
def get_sync_multiple_loc2(turn, list_speaker):
    who_tags = turn.find_all('Who')
    liste_text = []
    for i, who_tag in enumerate(who_tags): 
        contenu = []
        current_tag = who_tag.next_sibling
        while current_tag is not None and current_tag.name != 'Who': 
            if isinstance(current_tag, NavigableString): 
                current_tag_str = current_tag.get_text()
                contenu.append(fix_encoding(current_tag_str))
            else : 
                pass
            current_tag = current_tag.next_sibling
        liste_text.append(''.join(contenu).replace('\n',' '))
    tour = list(zip(list_speaker, liste_text))
    return tour

In [ ]:
df_meta.reset_index().iloc[5]['trs']

In [ ]:
list_df_turn= [parse_trs2(file) for file in df['trs'].tolist()]

In [ ]:
def get_autre(dict_meta, nom, L, tipe):
    try : 
        L = L['speaker_name'].replace('(ENQ)','').replace(' ', '').lower()
        for x, y  in dict_meta['locuteur'].items():
            x = x.replace(' ', '').lower()
            if x == L : 
                tipe = y['{}'.format(tipe)]
                return tipe
    except:
        return 'NA'

In [ ]:
def add_meta(df_essai, dict_meta, dict_path):
    #try
    nom = df_essai['trs_filename'].tolist()[0]
    print('nom:', nom, type(nom))
    try : 
        dict_corpus = dict_meta['{}'.format(nom)]
        dict_loc = dict_corpus['locuteur']
        df_essai['sexe_speaker'] = df_essai.apply(lambda row: get_autre(dict_corpus, nom, row, 'sexe'), axis=1)
        df_essai['age_speaker'] = df_essai.apply(lambda row: get_autre(dict_corpus, nom, row, 'age au moment de l’enregistrement'), axis=1)
        df_essai['profession_speaker'] = df_essai.apply(lambda row: get_autre(dict_corpus, nom, row, 'activité actuelle'), axis=1)
        df_essai['appartenance_speaker'] = df_essai.apply(lambda row: get_autre(dict_corpus, nom, row, "mobilité géographique"), axis=1)
        df_essai['autre_langue_speaker'] = df_essai.apply(lambda row: get_autre(dict_corpus, nom, row, 'langues parlées par le locuteur'), axis=1)
        df_essai['etude_speaker'] = df_essai.apply(lambda row: get_autre(dict_corpus, nom, row, "dernier diplôme obtenu"), axis=1)

        duree = dict_corpus['durée']
        date_enregistrement = dict_corpus['date']
        langue_enregistrement = 'français'
        lieu_enregistrement = 'FRANCE - Région Parisienne - '+ str(dict_corpus['quartier concerné'])

        df_essai['description'] = 'NA'
        df_essai['duree'] = duree 
        df_essai['responsable'] = 'NA'
        df_essai['date_enregistrement'] = date_enregistrement 
        df_essai['langue_enregistrement'] = langue_enregistrement 
        df_essai['lieu_enregistrement'] = lieu_enregistrement 
        
    except KeyError:
        df_essai['sexe_speaker'] = 'NA'
        df_essai['age_speaker'] = 'NA'
        df_essai['profession_speaker'] = 'NA'
        df_essai['appartenance_speaker'] = 'NA'
        df_essai['autre_langue_speaker'] = 'NA'
        df_essai['etude_speaker'] = 'NA'
        df_essai['description'] = 'NA'
        df_essai['duree'] = 'NA'
        df_essai['responsable'] = 'NA'
        df_essai['date_enregistrement'] = 'NA'
        df_essai['langue_enregistrement'] = 'NA'
        df_essai['lieu_enregistrement'] = 'NA'
        

    path_trs = str(dict_path['{}'.format(nom)]['corpus'])+'\\'+ nom
    path_wav = str(dict_path['{}'.format(nom)]['corpus'])

    df_essai['path_trs'] = path_trs
    df_essai['path_wav'] = df_essai['audio_filename'].apply(lambda x : path_wav +'\\'+x )
        

#         print('erreur dico : ', nom)
    #df_essai = df_essai.rename(columns={"corpus_name": "sous_corpus"})
    print('______________________________________')
    return df_essai
   # except:
    #     pass

In [ ]:
dict_meta = df_meta.set_index('trs').T.to_dict('dict')
dict_path = df.set_index('trs').T.to_dict('dict')

In [ ]:
final = [add_meta(df, dict_meta, dict_path) for df in list_df_turn]

In [ ]:
dict_path['Leo_Valentin_H_52_15e-v2.trs']

In [ ]:
df_tours = pd.concat(final)
df_tours['duree'] = df_tours['duree'].apply(lambda x : time2seconds(x))

In [ ]:
df_tours = pd.concat(final)
df_tours['duree'] = df_tours['duree'].apply(lambda x : time2seconds(x))
df_tours['corpus'] = ['CFPP']*len(df_tours)
df_tours = df_tours.drop('audio_filename', axis=1)
df_tours = df_tours.fillna('NA')
df_tours['encoding']='NA'
df_tours['sous_corpus'] = df_tours['trs_filename'].apply(lambda x : x.replace('.trs',''))
df_tours = df_tours.drop('trs_filename', axis=1)
df_tours['age_speaker'] = df_tours['age_speaker'].apply(lambda x : x.replace('et demi', '').replace('  (née en 1977) ',''))

In [ ]:
def time2seconds(string):
    try : 
        string = string.replace(' (deux fichiers de 53’58’’ et 34’20)','').replace(' heure ', 'h').replace(' ','')
        if "h" in string : 
            pattern2 = r'(\d+)h(\d*)'
            match2 = re.search(pattern2, string)
            if match2 : 
                liste = [int(match2.group(1)),int(match2.group(2)), 0]
        elif "minutes" in string:
            string = string.replace("minutes", '')
            liste = [0,int(string),0]

        else :    
            pattern = r'([0-9]*)([\S])([0-9]*)'
            match = re.search(pattern, string)
            if match : 
                if len(match.group(3))!=0 :
                    liste = [0,int(match.group(1)),int(match.group(3))]
                else: 
                    liste = [0,int(match.group(1)),0]

        ftr = [3600,60,1]
        seconds = sum([multi*value for multi,value in zip(ftr, liste)])
        return seconds
    except: 
        return 'NA'

In [ ]:
df_tours.to_csv('F:\Corpus\\finaux\CFPP.csv', sep='\t', encoding='utf-8', index=False)